### Лабораторная работа "“ЛАЗЕРНО-ФЛУОРЕСЦЕНТНАЯ СПЕКТРОСКОПИЯ МОЛЕКУЛ”"

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.signal import argrelextrema
from scipy.signal import find_peaks, peak_widths,savgol_filter
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
import copy
import math
from scipy.stats import t

In [2]:
data_path =  "spectroscopy_lab_data"
save_path = "results"

In [3]:
def unpack(path, mod): 
    file = open(path + "\\" + mod + ".txt", 'r' )
    data = file.readlines()
    data = [line.rstrip().split(",") for line in data]
    settings = data[:4]
    del data[:5]
    data = np.array(data).astype("float")
    time = data[:,0]
    signal = data[:, 1]
    return  settings, time, signal

mod = [ "spectrum", "background"]

In [ ]:
settings spectrum = 